In [1]:
import tensorflow as tf

import numpy as np
import random
import sys
import io

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pumkuan_path = tf.keras.utils.get_file('two-syllables.tsv', origin='https://drive.google.com/uc?export=download&id=119TGxoeDjLlFiZ4Tn-cdM57z3VUklYR8')
print pumkuan_path
with io.open(pumkuan_path, encoding='utf-8') as f:
    text = f.read().lower()

589824/578132 [==============================] - 0s 1us/step
/root/.keras/datasets/two-syllables.tsv


In [3]:
def preprocess_word(w):
    w = w.rstrip().strip()
    w = 's' + w + 'e'
    return w
# Create dataset
# Strip can also help remove the last empty line.
lines = text.strip().split('\n');
pairs = [[preprocess_word(w) for w in l.split('\t')] for l in lines]

In [4]:
chars = [unichr(x) for x in xrange(ord(u'\u0E01'), ord(u'\u0E4D'))]
# Add s and e (<start> and <end> token).
chars += [u's',u'e'];
# Create two-way indexes
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print char_indices

{u'\u0e01': 0, u'\u0e03': 2, u'\u0e02': 1, u'\u0e05': 4, u'\u0e04': 3, u'\u0e07': 6, u'\u0e06': 5, u'\u0e09': 8, u'\u0e08': 7, u'\u0e0b': 10, u'\u0e0a': 9, u'\u0e0d': 12, u'\u0e0c': 11, u'\u0e0f': 14, u'\u0e0e': 13, u'\u0e11': 16, u'\u0e10': 15, u'\u0e13': 18, u'\u0e12': 17, u'\u0e15': 20, u'\u0e14': 19, u'\u0e17': 22, u'\u0e16': 21, u'\u0e19': 24, u'\u0e18': 23, u'\u0e1b': 26, u'\u0e1a': 25, u'\u0e1d': 28, u'\u0e1c': 27, u'\u0e1f': 30, u'\u0e1e': 29, u'\u0e21': 32, u'\u0e20': 31, u'\u0e23': 34, u'\u0e22': 33, u'\u0e25': 36, u'\u0e24': 35, u'\u0e27': 38, u'\u0e26': 37, u'\u0e29': 40, u'\u0e28': 39, u'\u0e2b': 42, u'\u0e2a': 41, u'\u0e2d': 44, u'\u0e2c': 43, u'\u0e2f': 46, u'\u0e2e': 45, u'\u0e31': 48, u'\u0e30': 47, u'\u0e33': 50, u'\u0e32': 49, u'\u0e35': 52, u'\u0e34': 51, u'\u0e37': 54, u'\u0e36': 53, u'\u0e39': 56, u'\u0e38': 55, u'\u0e3b': 58, u'\u0e3a': 57, u'\u0e3d': 60, u'\u0e3c': 59, u'\u0e3f': 62, u'\u0e3e': 61, u'\u0e41': 64, u'\u0e40': 63, u'\u0e43': 66, u'\u0e42': 65, u'\u

In [8]:
# Vectorize the input and target answer.
input_tensor = [[char_indices[c] for c in original] for original, target in pairs]
target_tensor = [[char_indices[c] for c in target] for original, target in pairs]

In [3]:
path = tf.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))

614400/600901 [==============================] - 6s 10us/step
('corpus length:', 600893)
('total chars:', 57)


In [5]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [7]:
np.zeros((3,3,3), dtype=np.bool)

array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]]])

In [8]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [10]:
# build the model: a single LSTM
print('Build model...')
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(tf.keras.layers.Dense(len(chars)))
model.add(tf.keras.layers.Activation('softmax'))

optimizer = tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [16]:
# Good ref on temperature and softmax : https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally?newreg=9f3156c8e58a4539a612d57dbce9c524

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    print preds
    preds = np.log(preds) / temperature
    print preds
    exp_preds = np.exp(preds)
    print exp_preds
    preds = exp_preds / np.sum(exp_preds)
    print preds
    probas = np.random.multinomial(1, preds, 1)
    print probas
    print np.argmax(probas)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            break
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

In [17]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 169s 846us/step - loss: 1.4296
()
----- Generating text after Epoch: 0
('----- diversity:', 0.2)
----- Generating with seed: " good reason in the end to become
distru"
 good reason in the end to become
distru[3.87183917e-07 4.11998371e-05 5.40818543e-14 2.57676779e-12
 3.64768736e-11 1.98541392e-13 5.13068778e-15 2.87464168e-08
 5.43470650e-11 7.61369801e-10 2.87457037e-12 3.32110520e-11
 7.23557889e-12 1.55035349e-12 2.30140039e-12 3.93509959e-13
 2.16652304e-12 3.19817762e-12 1.31083827e-12 1.95331446e-12
 1.16224127e-14 2.04144715e-14 3.37851600e-12 1.14303573e-14
 2.14924571e-11 4.05854387e-12 2.26689861e-10 1.31470780e-03
 5.38675068e-03 4.40580957e-03 5.01233898e-03 1.07161030e-02
 2.04879208e-03 3.60511738e-04 1.86154625e-06 5.82796929e-04
 4.86737184e-09 3.02875293e-07 1.23531232e-02 1.65083911e-02
 3.02964565e-03 5.61680703e-04 1.00657181e-03 1.33387744e-07
 2.09313165e-03 9.27213073e-01 7.08655315e-03 1.57940758e-05


200285/200285 [==============================] - 168s 839us/step - loss: 1.4170
()
----- Generating text after Epoch: 1
('----- diversity:', 0.2)
----- Generating with seed: "a great deal depends upon whether they r"
a great deal depends upon whether they r[9.20888851e-06 2.78092572e-04 6.74909226e-11 4.45113324e-09
 2.44152876e-09 4.39578485e-10 1.97458619e-10 4.68997314e-07
 1.45191565e-08 2.01505983e-07 1.73385906e-11 8.64065416e-11
 1.11465323e-10 8.70239644e-11 4.11222889e-10 6.42471770e-11
 5.11248995e-11 8.79764178e-11 2.15939239e-10 2.17049601e-10
 4.96414437e-11 6.04978984e-11 6.25570706e-10 8.83106366e-11
 5.66836855e-10 5.95126101e-11 1.61345728e-08 1.69550404e-01
 3.07250203e-09 3.07635659e-08 2.28201770e-06 7.67301023e-01
 2.00234854e-06 7.62534782e-08 1.85851950e-05 2.40211729e-02
 1.65561698e-09 2.85032229e-06 5.36913394e-05 3.65484266e-06
 6.22336374e-05 2.57615112e-02 7.08159398e-09 6.28459196e-09
 4.74790606e-04 5.19901223e-05 1.24545168e-05 1.23821627e-02
 1.26485588

a great deal depends upon whether they r[9.20888851e-06 2.78092572e-04 6.74909226e-11 4.45113324e-09
 2.44152876e-09 4.39578485e-10 1.97458619e-10 4.68997314e-07
 1.45191565e-08 2.01505983e-07 1.73385906e-11 8.64065416e-11
 1.11465323e-10 8.70239644e-11 4.11222889e-10 6.42471770e-11
 5.11248995e-11 8.79764178e-11 2.15939239e-10 2.17049601e-10
 4.96414437e-11 6.04978984e-11 6.25570706e-10 8.83106366e-11
 5.66836855e-10 5.95126101e-11 1.61345728e-08 1.69550404e-01
 3.07250203e-09 3.07635659e-08 2.28201770e-06 7.67301023e-01
 2.00234854e-06 7.62534782e-08 1.85851950e-05 2.40211729e-02
 1.65561698e-09 2.85032229e-06 5.36913394e-05 3.65484266e-06
 6.22336374e-05 2.57615112e-02 7.08159398e-09 6.28459196e-09
 4.74790606e-04 5.19901223e-05 1.24545168e-05 1.23821627e-02
 1.26485588e-07 5.47500107e-08 1.37839129e-10 1.09181101e-05
 3.62950317e-12 3.77864025e-11 5.06180237e-11 4.69808394e-11
 4.67324755e-11]
[ -9.6627845   -6.82296376 -19.51585667 -16.02508926 -16.52553455
 -17.95433736 -18.62124

[1.87326271e-03 8.41562179e-01 1.59272614e-09 9.26834712e-09
 8.64779088e-12 4.69335300e-16 4.68059069e-11 5.65101845e-04
 7.69683659e-07 6.03944114e-06 1.18281540e-23 2.23573791e-21
 5.47157839e-23 2.18357138e-22 1.82228565e-22 9.85136839e-23
 4.26364322e-23 5.97691517e-22 1.58316909e-22 1.44196471e-22
 8.04230359e-09 1.15555203e-08 4.66965870e-22 1.98562326e-09
 5.12776581e-18 1.98660658e-18 6.27848712e-15 8.80334741e-08
 5.72369799e-11 3.98815064e-06 8.48613811e-09 7.62052596e-07
 8.50786063e-08 1.59391818e-12 9.38814623e-08 5.42215814e-08
 7.60447682e-12 1.10777052e-08 6.06211602e-07 1.49764568e-07
 2.85986355e-06 3.25476535e-07 2.14294709e-07 8.65273578e-13
 2.53672638e-09 2.14799290e-07 2.07931045e-07 1.01295757e-07
 1.05916921e-08 2.47959370e-10 1.94010803e-12 5.52386671e-06
 1.33358845e-12 2.31583209e-22 2.32445296e-22 2.05848708e-22
 3.14519430e-22]
[2.21944590e-03 9.97084777e-01 1.88706554e-09 1.09811587e-08
 1.02459222e-11 5.56069528e-16 5.54557447e-11 6.69533947e-04
 9.1192

200285/200285 [==============================] - 173s 864us/step - loss: 1.4033
()
----- Generating text after Epoch: 3
('----- diversity:', 0.2)
----- Generating with seed: "ctivity requires an agency that is activ"
ctivity requires an agency that is activ[1.25215342e-06 5.70208113e-06 8.10190032e-16 3.41376193e-07
 1.93954655e-14 4.63318782e-13 5.46477663e-15 4.27422613e-08
 5.84591966e-08 4.19249441e-10 1.03150778e-12 6.60475494e-11
 7.96725047e-11 2.05631475e-12 9.40023961e-11 6.29953424e-12
 3.17597749e-12 1.30533483e-11 1.31369212e-11 1.27205304e-11
 1.12270753e-14 4.34170454e-14 3.85877559e-16 6.15465278e-15
 1.45324364e-10 1.77024354e-10 8.57847647e-12 1.16965644e-01
 7.13077167e-11 5.23580525e-07 1.13131694e-06 8.35841298e-01
 4.10291051e-07 4.57979127e-07 4.80931951e-04 3.84426117e-02
 1.98326314e-10 1.94290400e-11 1.35557770e-04 4.32723946e-09
 1.91994013e-06 4.03731829e-03 3.75633080e-09 6.57879513e-14
 6.04649540e-05 5.19306150e-06 3.26784544e-07 5.90270101e-05
 8.38924819

...

200285/200285 [==============================] - 181s 904us/step - loss: 12.5301
()
----- Generating text after Epoch: 18
('----- diversity:', 0.2)
----- Generating with seed: "f the soul, out of which those ideas for"
f the soul, out of which those ideas for[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.12715490e-05 0.00000000e+00
 0.00000000e+00 6.79566659e-17 8.07737931e-33 3.99955918e-23
 1.69130033e-14 3.09063243e-15 3.39685778e-22 2.39124389e-24
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.54111435e-31 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.000000

   128/200285 [..............................] - ETA: 2:52 - loss: 16.1181

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


 67200/200285 [=========>....................] - ETA: 2:06 - loss: 15.9615

KeyboardInterrupt: 